In [1]:
# -*- coding: utf-8 -*-
"""
Year 3 Project: Codons
Code to extract protein ID from table and 
retrieve the cds's from the Ensembl server.
Careful: loops through whole table so only use once on college computer!
Created on Mon Dec  9 17:54:22 2019
@author: Marianne Buat
"""
#import relevant libraries

import pandas as pd
import requests, sys #to communicate with Ensembl servers
import json 

xls_file = pd.ExcelFile('Table_EV1_mini.xlsx') # Import the excel file and call it xls_file
df = xls_file.parse('C. GenesMini') #import into pandas only one of the sheets  

db_len = len(df)
#Create an output dataframe with only the parts we are interested in
geneIDs = df.get('Gene ID')
CDS=pd.Series([]) #Create new series to contain the sequences, will then be added to output dataframe
actualID=pd.Series([])

maxQueryLen = 10 #actually 50
server = "https://rest.ensembl.org"
ext = "/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json","type":'cds'}


#Loop through huge table to retrieve all sequences
ind = 0
#while ind+maxQueryLen<=db_len:
gene_ids = geneIDs[ind:ind+maxQueryLen+1] #select 50 IDs from table
id_list=gene_ids.values.tolist() #convert dataframe slice to list
 #convert to json string for compatibility (could also have used str())

inputData = json.dumps({'ids':id_list,'type':['cds']*len(id_list)}) #dictionary with ids and output type cds for all; 

r = requests.post(server+ext, headers=headers, data=inputData) #request data from server        
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json() #read response from server
#decoded is a list of dictionaries
for k in range(len(id_list)):
 CDS[ind+k] = decoded[k]['seq'] #store the retrieved sequences
 actualID[ind+k] = decoded[k]['id']

#Create an output dataframe for the results

outputDF = pd.DataFrame({'Gene ID':geneIDs,'answer ID':actualID,'CDS':CDS})
print(outputDF.head())                                                                

# #Save to file
# writer = pd.ExcelWriter('CDSQueryOutput.xlsx')
# outputDF.to_excel(writer)
# writer.save()
# print('Output is written successfully to Excel File.')

           Gene ID        answer ID  \
0  ENSG00000000003  ENST00000373020   
1  ENSG00000000419  ENST00000612152   
2  ENSG00000000457  ENST00000614008   
3  ENSG00000000460  ENST00000371588   
4  ENSG00000000938  ENST00000371582   

                                                 CDS  
0  ATGGCGTCCCCGTCTCGGAGACTGCAGACTAAACCAGTCATTACTT...  
1  ATGCTAAAACTGTATGCAATGTTTCTGACTCTCGTTTTTTTGGTCG...  
2  ATGCTAAAACTGTATGCAATGTTTCTGACTCTCGTTTTTTTGGTCG...  
3  ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...  
4  ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...  


In [4]:
#code for CodonTable fonction 
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

def CodonTable (my_seq):
    CodonsDict = { 
    "TTT": 0, "TTC": 0, "TTA": 0, "TTG": 0, "CTT": 0, 
    "CTC": 0, "CTA": 0, "CTG": 0, "ATT": 0, "ATC": 0, 
    "ATA": 0, "ATG": 0, "GTT": 0, "GTC": 0, "GTA": 0, 
    "GTG": 0, "TAT": 0, "TAC": 0, "TAA": 0, "TAG": 0, 
    "CAT": 0, "CAC": 0, "CAA": 0, "CAG": 0, "AAT": 0, 
    "AAC": 0, "AAA": 0, "AAG": 0, "GAT": 0, "GAC": 0, 
    "GAA": 0, "GAG": 0, "TCT": 0, "TCC": 0, "TCA": 0, 
    "TCG": 0, "CCT": 0, "CCC": 0, "CCA": 0, "CCG": 0, 
    "ACT": 0, "ACC": 0, "ACA": 0, "ACG": 0, "GCT": 0, 
    "GCC": 0, "GCA": 0, "GCG": 0, "TGT": 0, "TGC": 0, 
    "TGA": 0, "TGG": 0, "CGT": 0, "CGC": 0, "CGA": 0, 
    "CGG": 0, "AGT": 0, "AGC": 0, "AGA": 0, "AGG": 0, 
    "GGT": 0, "GGC": 0, "GGA": 0, "GGG": 0} 
    list_nucleotides =[my_seq[i:i+3] for i in range(0, len(my_seq), 3)]
    NumberCodons=0     
    for nucleotide in list_nucleotides:
        if not nucleotide in CodonsDict:
            continue

        else:
            CodonsDict[nucleotide]+= 1
            NumberCodons+=1
    return CodonsDict

print(CodonTable(my_seq))


{'TTT': 0, 'TTC': 0, 'TTA': 0, 'TTG': 0, 'CTT': 0, 'CTC': 0, 'CTA': 0, 'CTG': 0, 'ATT': 0, 'ATC': 1, 'ATA': 1, 'ATG': 0, 'GTT': 0, 'GTC': 0, 'GTA': 0, 'GTG': 0, 'TAT': 1, 'TAC': 0, 'TAA': 0, 'TAG': 0, 'CAT': 0, 'CAC': 0, 'CAA': 0, 'CAG': 0, 'AAT': 0, 'AAC': 0, 'AAA': 1, 'AAG': 0, 'GAT': 1, 'GAC': 0, 'GAA': 0, 'GAG': 0, 'TCT': 0, 'TCC': 0, 'TCA': 0, 'TCG': 1, 'CCT': 0, 'CCC': 0, 'CCA': 0, 'CCG': 0, 'ACT': 0, 'ACC': 0, 'ACA': 0, 'ACG': 0, 'GCT': 0, 'GCC': 1, 'GCA': 0, 'GCG': 0, 'TGT': 0, 'TGC': 0, 'TGA': 0, 'TGG': 1, 'CGT': 0, 'CGC': 0, 'CGA': 1, 'CGG': 0, 'AGT': 0, 'AGC': 0, 'AGA': 0, 'AGG': 0, 'GGT': 0, 'GGC': 0, 'GGA': 1, 'GGG': 0}


In [18]:

import pandas as pd
import requests, sys #to communicate with Ensembl servers
import json 

xls_file = pd.ExcelFile('CDSOutput.xlsx') # Import the excel file and call it xls_file
df = xls_file.parse() #import into pandas dataframe object  

db_len = len(df)
print(db_len)

gene_ids = df.get('CDS')#only the CDS column
output = pd.DataFrame()#empt7 df
for i in range(len(gene_ids)):
    seq = gene_ids[i]
    codon=CodonTable(seq)#use of CodontTable fonction that outputs a dictionary 
    output = output.append(codon, ignore_index=True)#append as rows with column the dictionary keys
    k=k+1
print(output.head())

df_merge_col = pd.merge(df, output, left_index=True, right_index=True)# use merge method, not join to add with respect to rows
df_merge_col.to_csv('CodonTableMini.csv',index=False)#use this method to save to csv, traditinal format, better than excel


34
    AAA  AAC   AAG  AAT  ACA  ACC  ACG  ACT  AGA  AGC  ...  TCG  TCT  TGA  \
0   5.0  4.0  10.0  9.0  3.0  6.0  1.0  8.0  5.0  4.0  ...  0.0  5.0  0.0   
1   4.0  4.0   6.0  6.0  3.0  1.0  1.0  3.0  4.0  2.0  ...  0.0  2.0  0.0   
2   4.0  3.0   7.0  5.0  2.0  2.0  1.0  3.0  4.0  2.0  ...  0.0  1.0  0.0   
3  13.0  5.0   5.0  5.0  4.0  1.0  0.0  6.0  6.0  3.0  ...  1.0  5.0  0.0   
4  13.0  5.0   5.0  5.0  4.0  1.0  0.0  6.0  6.0  3.0  ...  1.0  5.0  0.0   

   TGC  TGG  TGT  TTA  TTC   TTG   TTT  
0  4.0  4.0  7.0  1.0  5.0   4.0  12.0  
1  2.0  1.0  4.0  0.0  1.0   3.0   6.0  
2  2.0  1.0  5.0  0.0  2.0   3.0   7.0  
3  0.0  2.0  1.0  6.0  4.0   9.0   6.0  
4  1.0  4.0  2.0  6.0  4.0  11.0   6.0  

[5 rows x 64 columns]
